#### Import

In [1]:
#!/usr/bin/env python3
import sys
sys.path.insert(0,'..')
import urx
import time
import numpy as np
from math import *
import math3d as m3d
ur = urx.Robot("192.168.1.10",use_rt=True)
ur.set_tcp((0, 0, 0, 0, 0, 0))
tcp_acc=0.1
tcp_vel=0.1
time.sleep(0.2)
pose = ur.get_pose()
print(pose)

importing urx from /high_speed_scooping


tried 11 times to find a packet in data, advertised packet size: -2, type: 3
Data length: 68
tried 11 times to find a packet in data, advertised packet size: -2, type: 3
Data length: 1092
tried 11 times to find a packet in data, advertised packet size: -2, type: 3
Data length: 1418


<Transform:
<Orientation: 
array([[-3.41458109e-05,  9.99999997e-01,  6.57742734e-05],
       [ 9.99999999e-01,  3.41472879e-05, -2.24546932e-05],
       [-2.24569391e-05,  6.57735066e-05, -9.99999998e-01]])>
<Vector: (-0.50498, 0.45003, 0.42373)>
>


In [2]:
from ddh import Gripper
gripper = Gripper("ddh_scooping")

reading gripper config...
connecting to odrive...
found left finger
found right fingers


#### testing

In [ ]:
gripper.disarm()

In [ ]:
gripper.arm(150)
gripper.set_parallel_jaw(10, 0)
time.sleep(0.5)
a2_init = (gripper.left_a2 + gripper.right_a2)/2 

In [ ]:
gripper.set_stiffness(50)

In [ ]:
gripper.set_left_a1_phi(-5, 20)
gripper.set_right_a1_phi(5, -20)

In [ ]:
gripper.set_left_a1_phi(-0, 20)
gripper.set_right_a1_phi(0, -20)

In [ ]:
print(gripper.left_a2,gripper.right_a2)

In [ ]:
# check finger collision
while 1:
    a2_cur = (gripper.left_a2 + gripper.right_a2)/2
#     print(a2_cur)
    if a2_cur - a2_init > 0.5:
        print ("Collision detected!")
        break

In [ ]:
print(ur.getl())

In [ ]:
init = (-0.73, 0.17, 0.4, pi, 0, 0)
ur.movel(init, tcp_acc, tcp_vel)

In [ ]:
smack_vel = 100
smack_acc = 0.3
stop_acc = 0.5 #0.5
ur.speedl([0,0,-smack_vel,0,0,0],smack_acc,5)

In [ ]:
ur.speedl([0,0,smack_vel,0,0,0],stop_acc,5)
time.sleep(0.5)
ur.stopl(stop_acc)

In [100]:
gripper.set_left_a1_phi(-0, 20)
gripper.set_right_a1_phi(0, -20)

#### Experiment

In [121]:
gripper.arm(pos_gain = 100)
gripper.set_parallel_jaw(8, 0) #10,0
# init pose
init = m3d.Transform()
init.pos = (-0.505,0.45,0.45)
init.orient.rotate_xb(pi)
init.orient.rotate_zt(-pi/2)
ur.movel(init, tcp_acc, tcp_vel)
time.sleep(0.5)

In [122]:
# robot go down
spd_collide = 0
smack_vel = 0.4
smack_acc = 0.4
stop_smack_acc = 7
reverse_vel = 0.45
reverse_acc = 12 #0.5
stop_acc = 5
a2_init = (gripper.left_a2 + gripper.right_a2)/2 
ur.speedl([0,0,-smack_vel,0,0,0],smack_acc,5)
# robot detect collision, then stop, close gripper, and go up
while 1:
    a2_cur = (gripper.left_a2 + gripper.right_a2)/2
    #print(a2_cur)
    if a2_cur - a2_init > 0.3:
        print ("Collision detected!")
        spd_collide = ur.get_tcp_speed(wait=False)[2]
        gripper.set_left_a1_phi(-0, 12)
        gripper.set_right_a1_phi(0, -12)
#         ur.stopl(stop_smack_acc)
        ur.speedl([0,0,reverse_vel,0,0,0],reverse_acc,5)
        time.sleep(0.4)
        ur.stopl(stop_acc)
        break
print(spd_collide)

Collision detected!
-0.3863629280550919
